In [3]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification

ECE475 Frequentist Machine Learning <br>
Project 3: Model Assessment and Selection  <br> 
Ali Ghuman, Layth Yassin, Simon Yoon <br>

The project demonstates correct and incorrect use of k-fold cross-validation. One run uses the whole dataset to perform feature selection without augmentation, then uses cross-validation to test the KNN model. Because all of the data was used for feature selection, the test data influences the model's accuracy without validation. The second run also uses cross-validation prior, but selects the best features based on the unique training data for each fold in the cross-validation. This allows for feature selection without compromising the integrity of the test. The demonstration reveals the significant inaccuracies that can occur without proper model assessment and data usage.

In [145]:
RANGES = [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [18, 19], [19, 20], [20, 21], [21, 22], [22, 23], [23, 24], [24, 25], [25, 26], [26, 27], [27, 28], [28, 29], [29, 30], [30, 31], [31, 32], [32, 33], [33, 34], [34, 35], [35, 36], [36, 37], [37, 38], [38, 39], [39, 40], [40, 41], [41, 42], [42, 43], [43, 44], [44, 45], [45, 46], [46, 47], [47, 48], [48, 49], [49, 50]]
# print([ [i, i + 1] for i in range(50)])

def percent_accuracy(ds, model): # returns acc 
    X = ds[0]
    y = ds[1]
    y_hat = model.predict(X.reshape(1, -1))
    return y == y_hat
        
def correlation(ds): # returns corr of feature vs output
    X = ds[0]
    y = ds[1]
    df1 = pd.DataFrame(X)
    df2 = pd.DataFrame(np.array([y for i in range(len(X[0]))]).T)
    corr = df1.corrwith(df2)
    return corr

def best_predictors(ds, x): # return ind of x best predictors
    corr = pd.Series(correlation(ds))
    corr = corr.abs()
    best = corr.nlargest(x, 'all')
    best = best.index.values
    return best

def select_features(ds, best): # return augmented dataset with selected features
    X = np.transpose(ds[0])
    X_new = []
    for i in best:
        X_new.append(X[i])
    X = np.transpose(X_new)
    ds=[X, ds[1]]
    return ds

def split_data(ds, r): # return list of ranges of k subset
    X1 = np.delete(ds[0], r[0], axis = 0)
    y1 = np.delete(ds[1], r[0])
    ds1 = [X1, y1]
    
    X2 = ds[0][r[0]]
    y2 = [ds[1][r[0]]]
    ds2 = [X2, y2]
    return [ds1, ds2]

def test_model(ds, k, n, x=0): # returns average k-fold acc of knn model
    accuracy = 0
    for r in RANGES:
        data = split_data(ds, r)
        #right way
        if (x!=0):
            best = best_predictors(data[0], x) 
            ds1 = select_features(data[0], best)
            ds2 = select_features(data[1], best)
        #wrong way
        else:
            ds1 = data[0] 
            ds2 = data[1]
        model = KNeighborsClassifier(n_neighbors=n).fit(ds1[0], ds1[1])
        y_hat = model.predict(ds2[0].reshape(1, -1))
        accuracy += (ds2[1] == y_hat)[0] * 100 
    return (accuracy/k)

In [146]:
k = 50 # folds in cross-validation
n = 1 # nearest neighbors 
x = 100 # best predictor count
ds = make_classification(n_samples=50, n_features=5000) # test

print("The wrong way:")
best = best_predictors(ds, x)
ds_alt = select_features(ds, best)
accuracy = test_model(ds_alt, k, n, 0)
print("Accuracy: "+str(accuracy)+"%")
print()

print("The right way:")
accuracy = test_model(ds, k, n, x)
print("Accuracy: " + str(accuracy) + "%\n")

The wrong way:
Accuracy: 100.0%

The right way:
Accuracy: 64.0%

